# Home Credit Default Risk

Can you predict how capable each applicant is of repaying a loan?

Many people struggle to get loans due to **insufficient or non-existent credit histories**. And, unfortunately, this population is often taken advantage of by untrustworthy lenders.

Home Credit strives to broaden financial inclusion for the **unbanked population by providing a positive and safe borrowing experience**. In order to make sure this underserved population has a positive loan experience, Home Credit makes use of a variety of alternative data--including telco and transactional information--to predict their clients' repayment abilities.

While Home Credit is currently using various statistical and machine learning methods to make these predictions, they're challenging Kagglers to help them unlock the full potential of their data. Doing so will ensure that clients capable of repayment are not rejected and that loans are given with a principal, maturity, and repayment calendar that will empower their clients to be successful.

**Submissions are evaluated on area under the ROC curve between the predicted probability and the observed target.**

# Dataset

In [1]:
# #Python Libraries
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels
import pandas_profiling

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import time
import requests
import datetime

import missingno as msno
import math
import sys
import gc
import os

# #sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# #sklearn - metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score

# #XGBoost & LightGBM
import xgboost as xgb
import lightgbm as lgb

# #Missing value imputation
from fancyimpute import KNN, MICE

# #Hyperparameter Optimization
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

pd.options.display.max_columns = 150

c:\users\karti\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Dictionary

In [2]:
!ls -l ../data/

total 2621364
-rw-r--r-- 1 karti 197609  26567651 May 17 18:06 application_test.csv
-rw-r--r-- 1 karti 197609 166133370 May 17 18:06 application_train.csv
-rw-r--r-- 1 karti 197609 170016717 May 17 18:08 bureau.csv
-rw-r--r-- 1 karti 197609 375592889 May 17 18:08 bureau_balance.csv
-rw-r--r-- 1 karti 197609 424582605 May 17 18:10 credit_card_balance.csv
-rw-r--r-- 1 karti 197609     37436 May 26 21:49 HomeCredit_columns_description.csv
-rw-r--r-- 1 karti 197609 723118349 May 17 18:13 installments_payments.csv
-rw-r--r-- 1 karti 197609 392703158 May 17 18:14 POS_CASH_balance.csv
-rw-r--r-- 1 karti 197609 404973293 May 17 18:15 previous_application.csv
-rw-r--r-- 1 karti 197609    536202 May 17 18:06 sample_submission.csv


- application_{train|test}.csv

This is the main table, broken into two files for Train (**with TARGET**) and Test (without TARGET).
Static data for all applications. **One row represents one loan in our data sample.**

Observations:
* Each row is unique

-----


- bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

-----

- previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

-----

- installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.

![](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# Data Pre-processing

In [280]:
df_application_train = pd.read_csv("../data/application_train.csv")
df_application_test = pd.read_csv("../data/application_test.csv")
df_bureau = pd.read_csv("../data/bureau.csv")
df_bureau_balance = pd.read_csv("../data/bureau_balance.csv")
df_credit_card_balance = pd.read_csv("../data/credit_card_balance.csv")
df_installments_payments = pd.read_csv("../data/installments_payments.csv")
df_pos_cash_balance = pd.read_csv("../data/POS_CASH_balance.csv")
df_previous_application = pd.read_csv("../data/previous_application.csv")

In [281]:
print("df_application_train: ", df_application_train.shape)
print("df_application_test: ", df_application_test.shape)
print("df_bureau: ", df_bureau.shape)
print("df_bureau_balance: ", df_bureau_balance.shape)
print("df_credit_card_balance: ", df_credit_card_balance.shape)
print("df_installments_payments: ", df_installments_payments.shape)
print("df_pos_cash_balance: ", df_pos_cash_balance.shape)
print("df_previous_application: ", df_previous_application.shape)

df_application_train:  (307511, 122)
df_application_test:  (48744, 121)
df_bureau:  (1716428, 17)
df_bureau_balance:  (27299925, 3)
df_credit_card_balance:  (3840312, 23)
df_installments_payments:  (13605401, 8)
df_pos_cash_balance:  (10001358, 8)
df_previous_application:  (1670214, 37)


In [282]:
gc.collect()

505

## Feature: df_installments_payments

In [283]:
df_installments_payments['K_PREV_INSTALLMENT_PAYMENT_COUNT'] = df_installments_payments.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')
df_installments_payments['K_NUM_INSTALMENT_NUMBER_SUM'] = df_installments_payments.groupby('SK_ID_CURR')['NUM_INSTALMENT_NUMBER'].transform(np.sum)

df_installments_payments['TEMP_DAYS_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['DAYS_ENTRY_PAYMENT'].transform(np.sum)
df_installments_payments['K_DAYS_DIFF'] = df_installments_payments['TEMP_DAYS_INSTALMENT'] - df_installments_payments['TEMP_DAYS_ENTRY_PAYMENT']

df_installments_payments['TEMP_AMT_INSTALMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_INSTALMENT'].transform(np.sum)
df_installments_payments['TEMP_AMT_PAYMENT'] = df_installments_payments.groupby('SK_ID_CURR')['AMT_PAYMENT'].transform(np.sum)
df_installments_payments['K_AMT_DIFF'] = df_installments_payments['TEMP_AMT_INSTALMENT'] - df_installments_payments['TEMP_AMT_PAYMENT']

# #Drop Duplicates
df_installments_payments = df_installments_payments[['SK_ID_CURR', 'K_PREV_INSTALLMENT_PAYMENT_COUNT', 'K_NUM_INSTALMENT_NUMBER_SUM', 'K_DAYS_DIFF', 'K_AMT_DIFF']].drop_duplicates()

In [284]:
# #CHECKPOINT
print("df_installments_payments", df_installments_payments.shape)
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_installments_payments["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 291643 + 47944)

df_installments_payments (339587, 5)
291643
47944
Sum:  339587


## Feature: df_credit_card_balance

In [285]:
df_credit_card_balance['K_PREV_CREDIT_CARD_BALANCE_COUNT'] = df_credit_card_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_credit_card_balance['K_MONTHS_BALANCE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
df_credit_card_balance['K_MONTHS_BALANCE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)

df_credit_card_balance['TEMP_AMT_BALANCE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_BALANCE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_CREDIT_LIMIT_ACTUAL'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_UTILIZATION'] = df_credit_card_balance['TEMP_AMT_BALANCE']/df_credit_card_balance['TEMP_AMT_CREDIT_LIMIT_ACTUAL']
df_credit_card_balance['K_CREDIT_UTILIZATION_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.mean)
df_credit_card_balance['K_CREDIT_UTILIZATION_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.min)
df_credit_card_balance['K_CREDIT_UTILIZATION_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_UTILIZATION'].transform(np.max)

# #Validation: SK_ID_CURR = 105755
# #AMT_DRAWINGS_CURRENT = AMT_DRAWINGS_ATM_CURRENT + AMT_DRAWINGS_OTHER_CURRENT + AMT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.mean)
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_AMT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].transform(np.max)

df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_PAYMENT_TOTAL_CURRENT'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE'] = df_credit_card_balance.groupby('SK_ID_CURR')['AMT_TOTAL_RECEIVABLE'].transform(lambda x:x+1)
df_credit_card_balance['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'] = df_credit_card_balance['TEMP_AMT_PAYMENT_TOTAL_CURRENT']/df_credit_card_balance['TEMP_AMT_TOTAL_RECEIVABLE']
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.mean)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.min)
df_credit_card_balance['K_AMT_PAYMENT_OVER_RECEIVABLE_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['TEMP_AMT_PAYMENT_OVER_RECEIVABLE'].transform(np.max)

# #CNT_DRAWINGS_CURRENT = CNT_DRAWINGS_ATM_CURRENT + CNT_DRAWINGS_OTHER_CURRENT + CNT_DRAWINGS_POS_CURRENT
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MEAN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.mean)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MIN'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.min)
df_credit_card_balance['K_CNT_DRAWINGS_CURRENT_MAX'] = df_credit_card_balance.groupby('SK_ID_CURR')['CNT_DRAWINGS_CURRENT'].transform(np.max)


# #Drop Duplicates
df_credit_card_balance = df_credit_card_balance[['SK_ID_CURR', 'K_PREV_CREDIT_CARD_BALANCE_COUNT', 'K_MONTHS_BALANCE_MAX', 'K_MONTHS_BALANCE_MIN', 'K_CREDIT_UTILIZATION_MEAN', 'K_CREDIT_UTILIZATION_MIN', 'K_CREDIT_UTILIZATION_MAX', 'K_AMT_DRAWINGS_CURRENT_MEAN', 'K_AMT_DRAWINGS_CURRENT_MIN', 'K_AMT_DRAWINGS_CURRENT_MAX', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MEAN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MIN', 'K_AMT_PAYMENT_OVER_RECEIVABLE_MAX', 'K_CNT_DRAWINGS_CURRENT_MEAN', 'K_CNT_DRAWINGS_CURRENT_MIN', 'K_CNT_DRAWINGS_CURRENT_MAX']].drop_duplicates()

In [286]:
# #CHECKPOINT
print("df_credit_card_balance", df_credit_card_balance.shape)
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_train["SK_ID_CURR"]))))
print(len(set(df_credit_card_balance["SK_ID_CURR"]).intersection(set(df_application_test["SK_ID_CURR"]))))
print("Sum: ", 86905 + 16653)

df_credit_card_balance (103558, 16)
86905
16653
Sum:  103558


## Feature: df_pos_cash_balance

In [287]:
df_pos_cash_balance['K_PREV_POS_CASH_BALANCE_COUNT'] = df_pos_cash_balance.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.max)
df_pos_cash_balance['K_MONTHS_BALANCE_POS_CASH_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['MONTHS_BALANCE'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.max)
df_pos_cash_balance['K_CNT_INSTALMENT_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT'].transform(np.min)

df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MAX'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.max)
df_pos_cash_balance['K_CNT_INSTALMENT_FUTURE_MIN'] = df_pos_cash_balance.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].transform(np.min)

# #Drop Duplicates
df_pos_cash_balance = df_pos_cash_balance[['SK_ID_CURR', 'K_PREV_POS_CASH_BALANCE_COUNT', 'K_MONTHS_BALANCE_POS_CASH_MAX', 'K_MONTHS_BALANCE_POS_CASH_MIN', 'K_CNT_INSTALMENT_MAX', 'K_CNT_INSTALMENT_MIN', 'K_CNT_INSTALMENT_FUTURE_MAX', 'K_CNT_INSTALMENT_FUTURE_MIN']].drop_duplicates()

## Feature: df_previous_application

In [288]:
df_previous_application['K_PREV_PREVIOUS_APPLICATION_COUNT'] = df_previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].transform('count')

df_previous_application['K_AMT_ANNUITY_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_previous_application['K_AMT_ANNUITY_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_previous_application['K_AMT_ANNUITY_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)

df_previous_application['TEMP_CREDIT_ALLOCATED'] = df_previous_application['AMT_CREDIT']/df_previous_application['AMT_APPLICATION']
df_previous_application['K_CREDIT_ALLOCATED_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.mean)
df_previous_application['K_CREDIT_ALLOCATED_MAX'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.max)
df_previous_application['K_CREDIT_ALLOCATED_MIN'] = df_previous_application.groupby('SK_ID_CURR')['TEMP_CREDIT_ALLOCATED'].transform(np.min)

df_previous_application['K_AMT_DOWN_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.mean)
df_previous_application['K_AMT_DOWN_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.max)
df_previous_application['K_AMT_DOWN_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_DOWN_PAYMENT'].transform(np.min)

df_previous_application['K_AMT_GOODS_PRICE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.mean)
df_previous_application['K_AMT_GOODS_PRICE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.max)
df_previous_application['K_AMT_GOODS_PRICE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['AMT_GOODS_PRICE'].transform(np.min)

df_previous_application['K_DAYS_DECISION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.mean)
df_previous_application['K_DAYS_DECISION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.max)
df_previous_application['K_DAYS_DECISION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].transform(np.min)

df_previous_application['K_CNT_PAYMENT_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.mean)
df_previous_application['K_CNT_PAYMENT_MAX'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.max)
df_previous_application['K_CNT_PAYMENT_MIN'] = df_previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DRAWING_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.mean)
df_previous_application['K_DAYS_FIRST_DRAWING_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.max)
df_previous_application['K_DAYS_FIRST_DRAWING_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DRAWING'].transform(np.min)

df_previous_application['K_DAYS_FIRST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_FIRST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.max)
df_previous_application['K_DAYS_FIRST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_FIRST_DUE'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_1ST_VERSION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE_1ST_VERSION'].transform(np.min)

df_previous_application['K_DAYS_LAST_DUE_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.mean)
df_previous_application['K_DAYS_LAST_DUE_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.max)
df_previous_application['K_DAYS_LAST_DUE_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_LAST_DUE'].transform(np.min)

df_previous_application['K_DAYS_TERMINATION_MEAN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.mean)
df_previous_application['K_DAYS_TERMINATION_MAX'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.max)
df_previous_application['K_DAYS_TERMINATION_MIN'] = df_previous_application.groupby('SK_ID_CURR')['DAYS_TERMINATION'].transform(np.min)


# #Drop Duplicates
df_previous_application = df_previous_application[['SK_ID_CURR', 'K_PREV_PREVIOUS_APPLICATION_COUNT', 
                                          'K_AMT_ANNUITY_MEAN', 'K_AMT_ANNUITY_MAX', 'K_AMT_ANNUITY_MIN',
                                          'K_CREDIT_ALLOCATED_MEAN', 'K_CREDIT_ALLOCATED_MAX', 'K_CREDIT_ALLOCATED_MIN',
                                          'K_AMT_DOWN_PAYMENT_MEAN', 'K_AMT_DOWN_PAYMENT_MAX', 'K_AMT_DOWN_PAYMENT_MIN',
                                          'K_AMT_GOODS_PRICE_MEAN', 'K_AMT_GOODS_PRICE_MAX', 'K_AMT_GOODS_PRICE_MIN',
                                          'K_DAYS_DECISION_MEAN', 'K_DAYS_DECISION_MAX', 'K_DAYS_DECISION_MIN',
                                          'K_CNT_PAYMENT_MEAN', 'K_CNT_PAYMENT_MAX', 'K_CNT_PAYMENT_MIN',
                                          'K_DAYS_FIRST_DRAWING_MEAN', 'K_DAYS_FIRST_DRAWING_MAX', 'K_DAYS_FIRST_DRAWING_MIN',
                                          'K_DAYS_FIRST_DUE_MEAN', 'K_DAYS_FIRST_DUE_MAX', 'K_DAYS_FIRST_DUE_MIN',
                                          'K_DAYS_LAST_DUE_1ST_VERSION_MEAN', 'K_DAYS_LAST_DUE_1ST_VERSION_MAX', 'K_DAYS_LAST_DUE_1ST_VERSION_MIN',
                                          'K_DAYS_LAST_DUE_MEAN', 'K_DAYS_LAST_DUE_MAX', 'K_DAYS_LAST_DUE_MIN',
                                          'K_DAYS_TERMINATION_MEAN', 'K_DAYS_TERMINATION_MAX', 'K_DAYS_TERMINATION_MIN']].drop_duplicates()

## Feature: df_bureau_balance

In [289]:
# df_bureau_balance['K_BUREAU_BALANCE_COUNT'] = df_bureau_balance.groupby('SK_ID_BUREAU')['SK_ID_BUREAU'].transform('count')

df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.mean)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.max)
df_bureau_balance['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].transform(np.min)

# #Drop Duplicates
df_bureau_balance = df_bureau_balance[['SK_ID_BUREAU',
                                                 'K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN',
                                                 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX',
                                                'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [290]:
len(df_bureau_balance['SK_ID_BUREAU'].unique())

817395

## Feature: df_bureau

In [291]:
len(df_bureau["SK_ID_BUREAU"].unique())

1716428

In [292]:
df_bureau.shape

(1716428, 17)

In [293]:
df_bureau = pd.merge(df_bureau, df_bureau_balance, on="SK_ID_BUREAU", how="left", suffixes=('_bureau', '_bureau_balance'))

In [294]:
df_bureau['K_BUREAU_COUNT'] = df_bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].transform('count')

df_bureau['K_BUREAU_DAYS_CREDIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].transform(np.min)

df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_CREDIT_DAY_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.max)
df_bureau['K_BUREAU_DAYS_CREDIT_ENDDATE_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].transform(np.min)

df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.mean)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MAX'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.max)
df_bureau['K_BUREAU_DAYS_ENDDATE_FACT_MIN'] = df_bureau.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].transform(np.min)


df_bureau['K_BUREAU_CNT_CREDIT_PROLONG_MAX'] = df_bureau.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].transform(np.max)

# #To-Do: Calculate a utilization metric for some of the features below?
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].transform(np.min)

df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.mean)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.max)
df_bureau['K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].transform(np.min)

df_bureau['K_BUREAU_AMT_ANNUITY_MEAN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.mean)
df_bureau['K_BUREAU_AMT_ANNUITY_MAX'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.max)
df_bureau['K_BUREAU_AMT_ANNUITY_MIN'] = df_bureau.groupby('SK_ID_CURR')['AMT_ANNUITY'].transform(np.min)

# #Added from df_bureau_balance
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN'].transform(np.mean)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MAX'].transform(np.max)
df_bureau['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'] = df_bureau.groupby('SK_ID_CURR')['K_BUREAU_BALANCE_MONTHS_BALANCE_MIN'].transform(np.min)


# #Drop Duplicates
df_bureau = df_bureau[['SK_ID_CURR', 'K_BUREAU_COUNT',
                                      'K_BUREAU_DAYS_CREDIT_MEAN', 'K_BUREAU_DAYS_CREDIT_MAX', 'K_BUREAU_DAYS_CREDIT_MIN',
                                      'K_BUREAU_CREDIT_DAY_OVERDUE_MEAN', 'K_BUREAU_CREDIT_DAY_OVERDUE_MAX', 'K_BUREAU_CREDIT_DAY_OVERDUE_MIN',
                                      'K_BUREAU_DAYS_CREDIT_ENDDATE_MEAN', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MAX', 'K_BUREAU_DAYS_CREDIT_ENDDATE_MIN',
                                      'K_BUREAU_DAYS_ENDDATE_FACT_MEAN', 'K_BUREAU_DAYS_ENDDATE_FACT_MAX', 'K_BUREAU_DAYS_ENDDATE_FACT_MIN',
                                      'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_MAX_OVERDUE_MIN',
                                      'K_BUREAU_CNT_CREDIT_PROLONG_MAX',
                                      'K_BUREAU_AMT_CREDIT_SUM_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_MAX', 'K_BUREAU_AMT_CREDIT_SUM_MIN',
                                      'K_BUREAU_AMT_CREDIT_SUM_DEBT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_DEBT_MIN',
                                      'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MAX', 'K_BUREAU_AMT_CREDIT_SUM_LIMIT_MIN',
                                      'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MEAN', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MAX', 'K_BUREAU_AMT_CREDIT_SUM_OVERDUE_MIN',
                                      'K_BUREAU_AMT_ANNUITY_MEAN', 'K_BUREAU_AMT_ANNUITY_MAX', 'K_BUREAU_AMT_ANNUITY_MIN',
                      'K_BUREAU_BALANCE_MONTHS_BALANCE_MEAN', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MAX', 'K_BUREAU_BALANCE_MONTHS_BALANCE_MIN']].drop_duplicates()

In [298]:
gc.collect()

203

# Combine Datasets

## Encode categorical columns

In [299]:
arr_categorical_columns = df_application_train.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_train[var_col] = df_application_train[var_col].astype('category').cat.codes
gc.collect()

arr_categorical_columns = df_application_test.select_dtypes(['object']).columns
for var_col in arr_categorical_columns:
    df_application_test[var_col] = df_application_test[var_col].astype('category').cat.codes
gc.collect()

# arr_categorical_columns = df_credit_card_balance.select_dtypes(['object']).columns
# for var_col in arr_categorical_columns:
#     df_credit_card_balance[var_col] = df_credit_card_balance[var_col].astype('category').cat.codes

112

## Combine Datasets

### df_installments_payments

In [300]:
df_installments_payments_train = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_installments_payments_test = df_installments_payments[df_installments_payments["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [301]:
df_application_train = pd.merge(df_application_train, df_installments_payments_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))
df_application_test = pd.merge(df_application_test, df_installments_payments_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_installments_payments'))

### df_credit_card_balance

In [302]:
df_credit_card_balance_train = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_credit_card_balance_test = df_credit_card_balance[df_credit_card_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [303]:
df_application_train = pd.merge(df_application_train, df_credit_card_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))
df_application_test = pd.merge(df_application_test, df_credit_card_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_credit_card_balance'))

### df_pos_cash_balance

In [304]:
df_pos_cash_balance_train = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_pos_cash_balance_test = df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [305]:
df_application_train = pd.merge(df_application_train, df_pos_cash_balance_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))
df_application_test = pd.merge(df_application_test, df_pos_cash_balance_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_pos_cash_balance'))

### df_previous_application

In [306]:
df_previous_application_train = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_previous_application_test = df_previous_application[df_previous_application["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [307]:
df_application_train = pd.merge(df_application_train, df_previous_application_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))
df_application_test = pd.merge(df_application_test, df_previous_application_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_previous_application'))

### df_bureau_balance and df_bureau

In [311]:
df_bureau_train = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_train["SK_ID_CURR"])]
df_bureau_test = df_bureau[df_bureau["SK_ID_CURR"].isin(df_application_test["SK_ID_CURR"])]

In [312]:
df_application_train = pd.merge(df_application_train, df_bureau_train, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))
df_application_test = pd.merge(df_application_test, df_bureau_test, on="SK_ID_CURR", how="outer", suffixes=('_application', '_bureau'))

In [313]:
gc.collect()

240

# Model Building

## Train-Validation Split

In [314]:
input_columns = df_application_train.columns
input_columns = input_columns[input_columns != 'TARGET']
target_column = 'TARGET'

X = df_application_train[input_columns]
y = df_application_train[target_column]
gc.collect()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [315]:
xgb_params = {
    'seed': 42,
    'booster': 'gbtree',   #Tree-based models
    'silent': 0,           #Messages would be printed
    'n_thread': -1,        #-1: all cores are used
    'objective': 'binary:logistic',
    'eval_metric': 'auc', 

    'eta': 0.15000000000000002,            #Learning rate- makes the model more robust via shrinkage
    'min_child_weight': 12, #Tradeoff b/n over and underfitting
    'max_depth': 5,        #Tradeoff b/n over and underfitting
    'subsample': 0.8500000000000001,      #Fraction of observations to be randomly sampled for each tree
    'alpha': 20.0, 
    'colsample_bytree': 0.6000000000000001,
    'gamma': 0.85, 
    'lambda': 1.4,
    'n_estimators': 270
    
}

In [290]:
def score(params):
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    watchlist = [(dtrain, 'train'), (dtest, 'valid')]
    
    xgb_model = xgb.train(xgb_params, dtrain, 270, evals=watchlist, verbose_eval=100)
    predictions = xgb_model.predict(dtest, ntree_limit=xgb_model.best_iteration)
    
    score = roc_auc_score(y_test, predictions)
    print("\tScore {0}\n\n".format(score))
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}

In [291]:
def optimize(evals, cores=-1, optimizer=tpe.suggest, random_state=0):
    
    space = {
        'n_estimators': hp.quniform('n_estimators', 50, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.25, 0.025), 
        'max_depth':  hp.choice('max_depth', np.arange(1, 20, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'alpha' :  hp.quniform('alpha', 0, 20, 1),
        'lambda': hp.quniform('lambda', 1, 2, 0.1),
        'nthread': -1,
        'objective': 'binary:logistic',
        'booster': 'gbtree',
        'eval_metric': 'auc',
        'seed': random_state
    }
    
    best = fmin(score, space, algo=tpe.suggest, max_evals=evals)
    return best

In [292]:
best_param = optimize(evals = 3, optimizer=tpe.suggest)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)


[0]	train-auc:0.692237	valid-auc:0.687399
[100]	train-auc:0.804466	valid-auc:0.774894
[200]	train-auc:0.825914	valid-auc:0.77782
[269]	train-auc:0.837815	valid-auc:0.778323
	Score 0.7783638522085083


[0]	train-auc:0.692237	valid-auc:0.687399
[100]	train-auc:0.804466	valid-auc:0.774894
[200]	train-auc:0.825914	valid-auc:0.77782
[269]	train-auc:0.837815	valid-auc:0.778323
	Score 0.7783638522085083


[0]	train-auc:0.692237	valid-auc:0.687399
[100]	train-auc:0.804466	valid-auc:0.774894
[200]	train-auc:0.825914	valid-auc:0.77782
[269]	train-auc:0.837815	valid-auc:0.778323
	Score 0.7783638522085083


------------------------------------
The best hyperparameters are:  

{'alpha': 2.0, 'colsample_bytree': 0.75, 'eta': 0.1, 'gamma': 0.5, 'lambda': 1.2000000000000002, 'max_depth': 8, 'min_child_weight': 19.0, 'n_estimators': 941.0, 'subsample': 0.8500000000000001}


In [17]:
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(xgb_params, xgb.DMatrix(X_train, y_train), 270, watchlist, maximize=True, verbose_eval=100)

[0]	train-auc:0.727403	valid-auc:0.713529
[100]	train-auc:0.873175	valid-auc:0.754423
[200]	train-auc:0.922062	valid-auc:0.753007
[269]	train-auc:0.944288	valid-auc:0.750696


In [316]:
# #Final Model
gc.collect()
watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_test, y_test), 'valid')]
model = xgb.train(xgb_params, xgb.DMatrix(X, y), 270, watchlist, maximize=True, verbose_eval=100)

[0]	train-auc:0.667336	valid-auc:0.663777
[100]	train-auc:0.804673	valid-auc:0.804159
[200]	train-auc:0.824678	valid-auc:0.823006
[269]	train-auc:0.836008	valid-auc:0.833985


In [317]:
df_predict = model.predict(xgb.DMatrix(df_application_test), ntree_limit=model.best_ntree_limit)

In [318]:
submission = pd.DataFrame()
submission["SK_ID_CURR"] =  df_application_test["SK_ID_CURR"]
submission["TARGET"] =  df_predict

submission.to_csv("../submissions/model_1_xgbstarter_updatedParams_v6.csv", index=False)

In [319]:
# #Should be 48744
submission.shape

(48744, 2)

In [25]:
def score(params):

    # #Reference: http://sbern.com/2017/05/03/optimizing-hyperparameters-with-hyperopt/
    scores = []
    
    kf = StratifiedKFold(n_folds=20, random_state=RANDOM_STATE, shuffle = True)
    for i, (train_index, test_index) in enumerate(kf.split(X, y)):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]    
        
        xgb_train = xgb.DMatrix(X_train, label=y_train)
        xgb_val = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(xgb_train, 'train'), (xgb_val, 'eval')]
        xgb_model = xgb.train(xgb_params, xgb_train, 1000, watchlist, verbose_eval=200) 
        
        lg = xgb_model.predict(xgb_val, ntree_limit=model.best_ntree_limit) 
        res = [np.argmax(lg[i]) for i in range(lg.shape[0])]
        scores.append(accuracy_score(y_val, res))
        print('XGBoost', scores[-1])
    
    score = np.mean(scores)
    
    print("############### Score: {0}".format(score))
    print("############### Prms: ", params)
    print('..........................')
    
    score = roc_auc_score(y_test, predictions)
    print("\tScore {0}\n\n".format(score))
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}
